# 1. KLUE(Korean Language Understanding Evaluation)
- 한국어 자연어 이해 평가 데이터셋
- 한국어 언어모델의 공정한 평가를 위한 목적으로 8개의 종류가 포함된 공개 데이터셋
  - 뉴스 헤드라인 분류
  - 문장 유사도 비교
  - 자연어 추론
  - 개체명 인식
  - 관계 추출
  - 형태소 및 의존 구문 분석
  - 기계 독해 이해
  - 대화 상태 추적


# 2. 학습 데이터
- 광범위한 주제와 다양한 스타일을 포괄하기 위해 다양한 출처에서 공개적으로 사용 가능한 한국어 말뭉치를 수집
- 약 62GB크기의 최종 사전 학습 코퍼스를 구축
- MODU :국립국어원에서 배포하는 한국어 말뭉치 모음
- CC-100-Kor : CC-100은 CC-Net을 사용하여 대규모 다국어 웹 크롤링 코퍼스 구축
- 나무위키 : 나무위키는 한국어 웹 기반 백과사전으로 위키백과와 유사하지만 자유로운 형식으로 알려져있음(2020년 3월 2일에 생성된 덤프)
- 뉴스스크롤 : 2011년부터 2020년까지 발행한 12,800,000개의 뉴스기사로 구성되어 있어며, 뉴스 집계 플랫폼에서 수집
- 청원 : 사회적 이슈에 대한 행정 조치를 요청하는 청와대 국민청원 모음. 2017년 8월부터 2019년 3월 까지 게시된 청와대 국민청원의 기사 사용

# 3. KLUE-TC Task
- Topic Classification
- 주어진 뉴스 포제가 어떤 토픽에 속하는지 분류


In [3]:
!pip install -U transformers datasets scipy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
 

In [4]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00


In [5]:
import datasets
import random
import pandas as pd
import numpy as np
from datasets import load_dataset, ClassLabel, load_metric
from IPython.display import display, HTML
from transformers import AutoTokenizer, pipeline, AutoModelForSequenceClassification, Trainer,TrainingArguments

In [6]:
# ['ynat', 'sts', 'nli', 'ner', 're', 'dp', 'mrc', 'wos']
model_checkpoint = 'klue/roberta-base'
batch_size = 64
# 감정분석(Yes / No ) : 주어진 문장에 대해 '예', '아니오' 로 답변하는 작업
task='ynat'

In [7]:
datasets = load_dataset('klue',task)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

In [9]:
# datasets에 train 데이터 인덱스 0~9까지 출력

for data in range(10):
  print(datasets['train']['date'][data])


2016.06.30. 오전 10:36
2016.05.08. 오전 5:25
2016.03.15. 오후 12:00
2017.02.28. 오전 9:54
2018.04.03. 오전 7:05
2016.07.18. 오전 9:46
2018.06.04 08:15
2018.01.17. 오전 10:14
2019.10.30. 오전 4:17
2019.06.28. 오후 5:42


In [10]:
# show_random_elements(dataset, num_examples=10)
# train에서 랜덤하게 매개변수에 전달된 갯수만큼 데이터프레임으로 변환하여 출력
def show_random_elements(dataset,num_examples=10):
  picks=[]

  for _ in range(num_examples):
    pick = random.randint(0, len(dataset)-1)
    while pick in picks:
      pick = random.randint(0, len(dataset)-1)
    picks.append(pick)
  df = pd.DataFrame(dataset[picks])
  print(df)

  for column, typ in dataset.features.items():
    # isinstance(): typ(타입) , ClassLabel 클래스의 인스턴스인지 확인 즉 클래스 라벨에 해당하는 같은 타입인지 확인
    if isinstance(typ,ClassLabel):
      df[column] = df[column].transform(lambda i: typ.names[i])
  return df

In [11]:
show_random_elements(datasets['train'])

                  guid                            title  label  \
0  ynat-v1_train_45011      3차원 인식 가능한 증강현실 게임 지원기술 나온다      0   
1  ynat-v1_train_24037       300부터 안시성까지…영화로 배우는 전쟁의 역사      3   
2  ynat-v1_train_20351    LGU 통해 도박 스팸 수백만건 전송…방통위 수사의뢰      2   
3  ynat-v1_train_24563  트럼프 멕시코 장벽설치 비용 안내면 이민자 송금 차단해야      4   
4  ynat-v1_train_08705          청송 영하 9.8도…대구·경북 쌀쌀한 아침      3   
5  ynat-v1_train_08692                고미술 기획전 조선 병풍의 나라      3   
6  ynat-v1_train_43375     사이영상 고비 넘긴 류현진 선발투수라는 생각 지웠다      5   
7  ynat-v1_train_38628  비트코인 가격 변동성 달러화의 16배·美주식 대비 15배      1   
8  ynat-v1_train_15036  구글플레이 결제액 포인트로 쌓인다…1천원당 최대 2p종합      0   
9  ynat-v1_train_03380       올림픽파크 KT홍보관 5G커넥티드 방문한 최민정      0   

                                                 url                  date  
0  https://news.naver.com/main/read.nhn?mode=LS2D...  2016.08.02. 오전 11:39  
1  https://news.naver.com/main/read.nhn?mode=LS2D...   2019.01.11. 오전 6:00  
2  https://news.naver.com/main/read.nhn?mo

,guid,title,label,url,date
0,ynat-v1_train_45011,3차원 인식 가능한 증강현실 게임 지원기술 나온다,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.08.02. 오전 11:39
1,ynat-v1_train_24037,300부터 안시성까지…영화로 배우는 전쟁의 역사,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.11. 오전 6:00
2,ynat-v1_train_20351,LGU 통해 도박 스팸 수백만건 전송…방통위 수사의뢰,사회,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.02.13. 오후 4:47
3,ynat-v1_train_24563,트럼프 멕시코 장벽설치 비용 안내면 이민자 송금 차단해야,세계,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.04.06. 오전 3:19
4,ynat-v1_train_08705,청송 영하 9.8도…대구·경북 쌀쌀한 아침,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.02.08. 오전 8:01
5,ynat-v1_train_08692,고미술 기획전 조선 병풍의 나라,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.10.05. 오후 12:37
6,ynat-v1_train_43375,사이영상 고비 넘긴 류현진 선발투수라는 생각 지웠다,스포츠,https://sports.news.naver.com/news.nhn?oid=001...,2019.08.01 10:55
7,ynat-v1_train_38628,비트코인 가격 변동성 달러화의 16배·美주식 대비 15배,경제,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.07.06. 오후 12:00
8,ynat-v1_train_15036,구글플레이 결제액 포인트로 쌓인다…1천원당 최대 2p종합,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.04.23. 오전 11:53
9,ynat-v1_train_03380,올림픽파크 KT홍보관 5G커넥티드 방문한 최민정,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.02.24. 오후 3:33


In [12]:
metric = load_metric('f1')

<ipython-input-12-b37dd255292e>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('f1')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [13]:
fake_preds = np.random.randint(0,2,size=(64,))
fake_labels = np.random.randint(0,2,size=(64,))
fake_preds, fake_labels

(array([1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0]),
 array([0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1]))

In [14]:
metric.compute(predictions=fake_preds, references=fake_labels)

{'f1': 0.4126984126984127}

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,use_fast=True)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [16]:
tokenizer('크리스토퍼 놀란, 한국계 셀린 송 감독 패스트 라이브즈 극찬 미묘하게 아름다운')

{'input_ids': [0, 27763, 9920, 16, 3629, 2418, 1272, 2466, 1288, 3871, 13867, 10439, 2228, 17452, 12928, 2205, 2318, 5212, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
tokenizer

BertTokenizerFast(name_or_path='klue/roberta-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [18]:
print(f"Sentence 1: {datasets['train'][0]['title']}")

Sentence 1: 유튜브 내달 2일까지 크리에이터 지원 공간 운영


In [19]:
def preprocess_function(examples):
  return tokenizer(
      examples['title'],
      truncation=True,
      return_token_type_ids=False
  )

In [20]:
preprocess_function(datasets['train'][:5])

{'input_ids': [[0, 10637, 8474, 22, 2210, 2299, 2118, 28940, 3691, 4101, 3792, 2], [0, 24905, 1042, 4795, 19982, 2129, 121, 6904, 16311, 3, 14392, 2], [0, 4172, 3797, 3728, 2107, 2134, 3777, 904, 6022, 2332, 2113, 2259, 4523, 1380, 2259, 2062, 2], [0, 12417, 2155, 7840, 604, 2859, 3873, 11554, 2522, 1539, 2073, 8446, 6626, 18818, 575, 2], [0, 13203, 2179, 2366, 4197, 7551, 2096, 8542, 2088, 2353, 886, 1244, 4393, 2027, 22, 2207, 8189, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [21]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [22]:
num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# averate='macro' : macro는 평가하려는 매트릭,
# 클래스마다 동일한 가중치를 적용하여 평균을 계산하는 방법
# 평균을 클래스별로 각각의 평균을 계산하고, 이들의 평균을 구함
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels, average='macro')

In [24]:
metric_name = 'f1'

# evaluation_strategy : 평가전략, epoch : 에폭이 끝날 때 마다 평가를 수행
# save_strategy : 모델 체크포인트를 저장하는 전략. epoch : 에폭이 끝날 때마다 모델을 저장
# learning_rate : 학습률
# per_device_train_batch_size : 학습시 배치크기를 설정
# per_device_eval_batch_size : 평가시 배치크기를 설정
# num_train_epochs : 전체 학습 에폭의 수를 설정
# weight_decay : 가중치 감소의 설정
# load_best_model_at_end : 학습이 끝난 후 가장 좋은 성능을 보인 모델을 불러올지 여부를 지정
# metric_for_best_model : 최적의 모델을 결정할 떄 사용할 매트릭을 지정, F1
args = TrainingArguments(
    'test-tc',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay = 0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name

)

In [25]:
# Trainer : 모델의 학습과 평과를 관리하는 클래스
# 주어진 데이터셋으로부터 배치를 생성하고, 모델에 입력을 전단하여
# 손실을 계산하고 역전파를 수행함
trainer = Trainer(
    model,
    args,
    train_dataset = encoded_datasets['train'],
    eval_dataset = encoded_datasets['validation'],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.508600,0.389507,0.860971
2,0.327600,0.374645,0.868031
3,0.229200,0.389903,0.868555
4,0.199900,0.403431,0.867004
5,0.147400,0.425251,0.866590


TrainOutput(global_step=3570, training_loss=0.26338795306635837, metrics={'train_runtime': 1034.8328, 'train_samples_per_second': 220.702, 'train_steps_per_second': 3.45, 'total_flos': 2656603583505600.0, 'train_loss': 0.26338795306635837, 'epoch': 5.0})

In [27]:
trainer.evaluate()

{'eval_loss': 0.3899029791355133,
 'eval_f1': 0.8685547510465419,
 'eval_runtime': 11.5393,
 'eval_samples_per_second': 789.215,
 'eval_steps_per_second': 12.392,
 'epoch': 5.0}

In [28]:
classifier = pipeline(
    'text-classification',
    model = './text-tc/checkpoint-3570',
    return_all_scores=True
)

OSError: Incorrect path_or_model_id: './text-tc/checkpoint-3570'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
text =
'''
0 (IT과학)
1 (경제)
2 (사회)
3 (생활문화)
4 (세계)
5 (스포츠)
6 (정치)
'''

classifier('주호민 아들 학대 특수교사, 벌금형 선고유예 "미필적 고의 인정"(종합)')

In [ ]:
classifier('동점골 넣고도 최하 평점...英 매체, "5골에 모두 관여한 양날의 검"')